# Swimming Pool Detection With EfficientDet(Evaluation)

### Import libraries

In [1]:
!pip install pycocotools>=2.0.2 > /dev/null
!pip install timm>=0.3.2 > /dev/null
!pip install omegaconf>=2.0 > /dev/null
!pip install ensemble-boxes > /dev/null
!pip install effdet > /dev/null

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


<span style="color: #000508; font-family: Segoe UI; font-size: 2.0em; font-weight: 300;">Import Packages</span>

In [2]:
import sys
import torch
import os
import warnings
import time
import cv2
import random
import gc
import numba
import re
import ast

import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from datetime import datetime
from collections import Counter
from glob import glob

from ensemble_boxes import weighted_boxes_fusion
from albumentations.pytorch.transforms import ToTensorV2
import albumentations as A
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from numba import jit
from typing import List, Union, Tuple
from scipy.optimize import linear_sum_assignment

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold, train_test_split
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.utils.data.dataloader import default_collate

from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
from effdet import create_model, unwrap_bench, create_loader, create_dataset, create_evaluator, create_model_from_config
from effdet.data import resolve_input_config, SkipSubset
from effdet.anchors import Anchors, AnchorLabeler
from timm.models import resume_checkpoint, load_checkpoint
from timm.optim import create_optimizer
from timm.scheduler import create_scheduler

# Paths
TRAIN_ROOT_PATH = '../input/swimming-pool-512x512/CANNES_TILES_512x512_PNG/CANNES_TILES_512x512_PNG'
TRAIN_LABELS_PATH = '../input/k/alexj21/swimmingpool-eda-csv/swimming_pools_labels_512x512.csv'

# Models
NFOLDS = 5
MODELS_PATH = []
MODELS_PATH.append('../input/k/alexj21/efficientdet-swimming-pool-detection-custom/training_job/fold-0-best-checkpoint-039epoch.bin')
MODELS_PATH.append('../input/k/alexj21/efficientdet-swimming-pool-detection-custom/training_job/fold-1-best-checkpoint-033epoch.bin')
MODELS_PATH.append('../input/k/alexj21/efficientdet-swimming-pool-detection-custom/training_job/fold-2-best-checkpoint-030epoch.bin')
MODELS_PATH.append('../input/k/alexj21/efficientdet-swimming-pool-detection-custom/training_job/fold-3-best-checkpoint-039epoch.bin')
MODELS_PATH.append('../input/k/alexj21/efficientdet-swimming-pool-detection-custom/training_job/fold-4-best-checkpoint-031epoch.bin')

SEED = 42
IMG_SIZE = 512
PREDS_TH = 0.3

label2color = [[255, 0, 0]]
viz_labels =  ["pool"]

# Load data
df_annotations = pd.read_csv(TRAIN_LABELS_PATH)
df_annotations['image_path'] = df_annotations['image_id'].map(lambda x:os.path.join(TRAIN_ROOT_PATH, str(x)))

# Filter wrong annotations (or too small pools)
df_annotations = df_annotations.drop(df_annotations[(df_annotations['xmin'] == 0) & (df_annotations['xmax'] == 0)].index)
df_annotations = df_annotations.drop(df_annotations[(df_annotations['ymin'] == 0) & (df_annotations['ymax'] == 0)].index)

# Keep only pools
df_annotations = df_annotations[df_annotations['class'] == 'pool']
df_annotations.reset_index(drop=True, inplace=True)
df_annotations['class'] = 1
df_annotations['xmin'] = df_annotations['xmin'] - 1
df_annotations['ymin'] = df_annotations['ymin'] - 1
df_annotations.head(5)

,image_id,width,height,class,xmin,ymin,xmax,ymax,fold,nboxes,bbox_area,image_path
0,CANNES_TILES_512x512.1898.png,512,512,1,150,218,183,257,4,1,1216,../input/swimming-pool-512x512/CANNES_TILES_51...
1,CANNES_TILES_512x512.476.png,512,512,1,99,154,164,199,4,1,2816,../input/swimming-pool-512x512/CANNES_TILES_51...
2,CANNES_TILES_512x512.476.png,512,512,1,455,161,502,207,4,1,2070,../input/swimming-pool-512x512/CANNES_TILES_51...
3,CANNES_TILES_512x512.670.png,512,512,1,257,188,308,221,0,1,1600,../input/swimming-pool-512x512/CANNES_TILES_51...
4,CANNES_TILES_512x512.670.png,512,512,1,343,133,396,162,0,1,1456,../input/swimming-pool-512x512/CANNES_TILES_51...


In [3]:
image_paths = df_annotations['image_path'].unique()
print("Number of Images :",len(image_paths))
anno_count = df_annotations.shape[0]
print("Number of Annotations:", anno_count)

Number of Images : 1224
Number of Annotations: 3197


### Functions

In [4]:
def calc_stats(gt_boxes, pred_boxes, th=0.5):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            iou_score = calculate_iou(box1, box2)
            
            if iou_score < th:
                continue
            else:
                cost_matix[i,j]=0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy

    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area


def show_result(sample_id, preds, gt_boxes):
    sample = cv2.imread(f'{TRAIN_ROOT_PATH}/{sample_id}', cv2.IMREAD_COLOR)
    sample = cv2.cvtColor(sample, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for pred_box in preds:
        cv2.rectangle(
            sample,
            (pred_box[0], pred_box[1]),
            (pred_box[2], pred_box[3]),
            (220, 0, 0), 2
        )

    for gt_box in gt_boxes:    
        cv2.rectangle(
            sample,
            (gt_box[0], gt_box[1]),
            (gt_box[2], gt_box[3]),
            (0, 0, 220), 2
        )

    ax.set_axis_off()
    ax.imshow(sample)
    ax.set_title("RED: Predicted | BLUE - Ground-truth")

In [5]:
# Seed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Plots
def plot_img(img, size=(18, 18), is_rgb=True, title="", cmap=None):
    plt.figure(figsize=size)
    plt.imshow(img)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title="", cmap=None, img_size=None):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size), constrained_layout=True)
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.axis('off')
        plt.imshow(img)
    plt.suptitle(title)
    return fig

def draw_bbox_small(image, box, label, color):   
    alpha = 0
    alpha_text = 0.4
    thickness = 1
    font_size = 0.4
    overlay_bbox = image.copy()
    overlay_text = image.copy()
    output = image.copy()

    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, font_size, thickness)[0]
    cv2.rectangle(overlay_bbox, (box[0], box[1]), (box[2], box[3]),
                color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, (box[0], box[1]-7-text_height), (box[0]+text_width+2, box[1]),
                (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_text, output, 1 - alpha_text, 0, output)
    cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]),
                    color, thickness)
    cv2.putText(output, label.upper(), (box[0], box[1]-5),
            cv2.FONT_HERSHEY_SIMPLEX, font_size, (255, 255, 255), thickness, cv2.LINE_AA)
    return output
        
# Dataset class
class DatasetRetriever(Dataset):

    def __init__(self, marking, image_ids, transforms=None, test=False):
        super().__init__()
        self.image_ids = image_ids
        self.marking = marking
        self.transforms = transforms
        self.test = test
        
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        
        image, boxes, labels = self.load_image_and_boxes(index)
        
        ## To prevent ValueError: y_max is less than or equal to y_min for bbox from albumentations bbox_utils
        labels = np.array(labels, dtype=np.int).reshape(len(labels), 1)
        combined = np.hstack((boxes.astype(np.int), labels))
        combined = combined[np.logical_and(combined[:,2] > combined[:,0],
                                                          combined[:,3] > combined[:,1])]
        boxes = combined[:, :4]
        labels = combined[:, 4].tolist()
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = torch.tensor(labels)
        target['image_id'] = torch.tensor([index])
        if self.transforms:
            for i in range(10):
                sample = self.transforms(**{
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                })
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
                    target['boxes'][:,[0,1,2,3]] = target['boxes'][:,[1,0,3,2]]  ## ymin, xmin, ymax, xmax
                    break
            
            ## Handling case where no valid bboxes are present
            if len(target['boxes'])==0 or i==9:
                return None
            else:
                ## Handling case where augmentation and tensor conversion yields no valid annotations
                try:
                    assert torch.is_tensor(image), f"Invalid image type:{type(image)}"
                    assert torch.is_tensor(target['boxes']), f"Invalid target type:{type(target['boxes'])}"
                except Exception as E:
                    print("Image skipped:", E)
                    return None      

        return image, target, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]
    
    def load_image_and_boxes(self, index):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}', cv2.IMREAD_COLOR).copy()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        records = self.marking[self.marking['image_id'] == image_id]
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        labels = records['class'].tolist()
        resize_transform = A.Compose([A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0)], 
                                    p=1.0, 
                                    bbox_params=A.BboxParams(
                                        format='pascal_voc',
                                        min_area=0.1, 
                                        min_visibility=0.1,
                                        label_fields=['labels'])
                                    )

        resized = resize_transform(**{
                'image': image,
                'bboxes': boxes,
                'labels': labels
            })

        resized_bboxes = np.vstack((list(bx) for bx in resized['bboxes']))
        return resized['image'], resized_bboxes, resized['labels']
    
def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=IMG_SIZE, width=IMG_SIZE, p=1.0),
            ToTensorV2(p=1.0),
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0, 
            min_visibility=0,
            label_fields=['labels']
        )
    )


def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    
    return tuple(zip(*batch))
            
seed_everything(SEED)

### Iterate through images

In [6]:
warnings.filterwarnings("ignore")

models = [None] * NFOLDS
preds = [[] for j in range(NFOLDS)]

precision = [None] * NFOLDS
recall = [None] * NFOLDS
f1_score = [None] * NFOLDS

for fold in range(NFOLDS):
    # Validation ids
    val_ids = df_annotations[df_annotations['fold'] == fold].image_id.unique()

    # Creation validation dataset
    validation_dataset = DatasetRetriever(
                            image_ids=val_ids,
                            marking=df_annotations,
                            transforms=get_valid_transforms(),
                            test=True,
                            )

    # Create validation loader
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=4,
        num_workers=1,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
        collate_fn=collate_fn,
    )

    # Load and configure model
    base_config = get_efficientdet_config('tf_efficientdet_d1')
    base_config.image_size = (IMG_SIZE, IMG_SIZE)
    
    models[fold] = create_model_from_config(base_config, 
                                            bench_task='predict', 
                                            num_classes=1,
                                            pretrained=True)

    checkpoint = torch.load(MODELS_PATH[fold])
    models[fold].model.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()
    models[fold].eval()
    models[fold].cuda()
    
    # Loop through validation images
    print('Compute preds for fold', fold)
    ftp, ffp, ffn = [], [], []
    for images, targets, image_ids in tqdm(val_loader, total=len(val_loader)):
        with torch.no_grad():
            images = torch.stack(images)
            images = images.cuda().float()

            target_res = {}
            boxes = [target['boxes'].cuda().float() for target in targets]
            labels = [target['labels'].cuda().float() for target in targets]
            target_res['bbox'] = boxes
            target_res['cls'] = labels 
            target_res["img_scale"] = torch.tensor([1.0] * 8,
                                                   dtype=torch.float).cuda()
            target_res["img_size"] = torch.tensor([images[0].shape[-2:]] * 8,
                                                  dtype=torch.float).cuda()

            det = models[fold](images, target_res)

            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4].astype(int)    
                scores = det[i].detach().cpu().numpy()[:,4]
                gt_boxes = (targets[i]['boxes'][:,[1,0,3,2]]).cpu().numpy().astype(int)

                indexes = np.where(scores>0.3)
                pred_boxes = boxes[indexes]
                scores = scores[indexes]

                tp, fp, fn = calc_stats(gt_boxes, pred_boxes, th=0.3)
                ftp.append(tp)
                ffp.append(fp)
                ffn.append(fn)

    tp = np.sum(ftp)
    fp = np.sum(ffp)
    fn = np.sum(ffn)
    precision[fold] = tp / (tp + fp + 1e-6)
    recall[fold] =  tp / (tp + fn +1e-6)
    f1_score[fold] = 2*(precision[fold] *recall[fold])/(precision[fold] +recall[fold]+1e-6)
    
    print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision[fold] :.4f}, RECALL: {recall[fold]:.4f}, F1 SCORE: {f1_score[fold]:.4f} \n \n')
    
print(f'OOF -  PRECISION: {np.mean(precision) :.4f}, RECALL: {np.mean(recall):.4f}, F1 SCORE: {np.mean(f1_score):.4f}')

Downloading: "https://github.com/rwightman/efficientdet-pytorch/releases/download/v0.1/tf_efficientdet_d1_40-a30f94af.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientdet_d1_40-a30f94af.pth


Compute preds for fold 0



TP: 625, FP: 52, FN: 15, PRECISION: 0.9232, RECALL: 0.9766, F1 SCORE: 0.9491 
 

Compute preds for fold 1



TP: 607, FP: 53, FN: 33, PRECISION: 0.9197, RECALL: 0.9484, F1 SCORE: 0.9338 
 

Compute preds for fold 2



TP: 610, FP: 50, FN: 29, PRECISION: 0.9242, RECALL: 0.9546, F1 SCORE: 0.9392 
 

Compute preds for fold 3



TP: 608, FP: 50, FN: 31, PRECISION: 0.9240, RECALL: 0.9515, F1 SCORE: 0.9375 
 

Compute preds for fold 4



TP: 607, FP: 52, FN: 32, PRECISION: 0.9211, RECALL: 0.9499, F1 SCORE: 0.9353 
 

OOF -  PRECISION: 0.9224, RECALL: 0.9562, F1 SCORE: 0.9390
